# LCLS-II Exercise
### ***Special thanks to Irene Wang and Aditya Thapa for the LCLS-II input files!***

In [1]:
from genesis.version4 import Genesis4

## Scanning parameters

In the last notebook, we ran a base line LCLS-II simulation.
In this notebook, you will scan a parameter and plot how the performance changes.

<div class="alert alert-block alert-info">

Using the LCLS-II lattice file, try scanning some of the following:
- Peak current
- Energy spread
- Tapering

How does the performance change?
<div>

In [2]:
G = Genesis4("examples/lcls-II.in")
G.verbose = True
#G.input

In [3]:
output = G.run()

Configured to run in: /var/folders/qx/dyrxp4cj03l5328wsbdkpz20znqv3p/T/tmphhjf1qj3
Running Genesis4 in /var/folders/qx/dyrxp4cj03l5328wsbdkpz20znqv3p/T/tmphhjf1qj3
/Users/aaditya/opt/anaconda3/envs/fel2024/bin/genesis4 -l lcls-II.lat genesis4.in
---------------------------------------------
GENESIS - Version 4.6.6 has started...
Compile info: Compiled by runner at 2024-06-10 16:26:41 [UTC] from Git Commit ID: 
Starting Time: Thu Aug 15 14:42:48 2024

MPI-Comm Size: 1 node

Opened input file genesis4.in
Parsing lattice file lcls-II.lat ...
Matching for periodic solution between z = 0 and z = 9 :
   betax (m) : 2.94989
   alphax    : -0.4868
   phix (deg): 87.595
   betay (m) : 13.9016
   alphay    : 2.17265
   phiy (deg): 92.2785
Generating input radiation field for HARM = 1 ...
Generating input particle distribution...
Writing particle distribution to file: beginning.par.h5 ...
Writing particle distribution to file: end.par.h5 ...

Program is terminating...
Ending Time: Thu Aug 15 14:4

[PC100405:05402] shmem: mmap: an error occurred while determining whether or not /var/folders/qx/dyrxp4cj03l5328wsbdkpz20znqv3p/T//ompi.PC100405.1062988918/jf.0/1441333248/sm_segment.PC100405.1062988918.55e90000.0 could be created.


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/var/folders/qx/dyrxp4cj03l5328wsbdkpz20znqv3p/T/tmphhjf1qj3/lcls-II.out.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
output.lattice.plot()

In [ ]:
G.plot(["beam_xsize", "beam_ysize", "field_xsize", "field_ysize"])

In [ ]:
G.plot(["field_power"], yscale="log")

### Beam and field sizes

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
z = output.lattice.zplot
b = output.beam.bunching
p = output.field.power

fig, ax1 = plt.subplots()
color = "tab:red"
ax1.set_xlabel(r"$z$ (m)")
ax1.set_ylabel(r"$P$ (W)", color=color)
ax1.tick_params(axis="y", labelcolor=color)
ax1.semilogy(z, p, color=color)

ax2 = ax1.twinx()
color = "tab:blue"
ax2.set_ylabel(r"$<\exp(i\theta)>$", color=color)
ax2.tick_params(axis="y", labelcolor=color)
ax2.semilogy(z, b, color=color, alpha=0.5)
ax2.set_ylim([1e-4, 0.6])
plt.show()

In [ ]:
import numpy as np

In [ ]:
# Scan peak current

G_scan = G
current_list = np.linspace(500, 2000, 5)
scan_output = []

for current in current_list:
    G_scan.input.main.beam.current = current
    G_scan.verbose = False
    scan_output.append(G_scan.run())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
for k, g in zip(current_list, scan_output):
    x = g.stat("zplot")
    y = g.stat("power")
    ax.plot(x, y / 1e6, label=f"{k:0.1f}")
ax.set_yscale("log")
ax.set_xlabel(r"$z$ (m)")
ax.set_ylabel("power (MW)")
plt.legend(title=r"Peak Current $(A)$")

In [ ]:
fig, ax = plt.subplots()
y = np.array([g.stat("power")[-1] for g in scan_output])
ixbest = y.argmax()
Gbest = scan_output[ixbest]
current_best = current_list[ixbest]
ybest = y[ixbest]
ax.plot(current_list, y / 1e6)
ax.scatter(current_best, ybest / 1e6, marker="*", label=rf"$k$= {current_best:0.1f} 1/m$^2$")
ax.set_ylabel(r"end power $(MW)$")
ax.set_xlabel(r"Peak Current $(A)$")
plt.legend()

In [ ]:
G_scan.plot("power", yscale="log", y2=["beam_xsize", "beam_ysize"], ylim2=(0, 50e-6))

In [ ]:
# No taper

output.load_particles()
pbeg = output.particles['beginning']
pend = output.particles['end']

In [ ]:
pend.plot("z", "energy", bins = 200) # No taper

# Taper!

## Trying linear taper. Not very agressive. Check file **LCLS00-TAPER1.in** for lattice

In [ ]:
# try some taper

G_taper1 = Genesis4("examples/LCLSII-TAPER1.in")
G_taper1.verbose = True
G_taper1.input

In [ ]:
taper1 = G_taper1.run()

In [ ]:
taper1.lattice.plot();

In [ ]:
G_taper1.plot(["beam_xsize", "beam_ysize", "field_xsize", "field_ysize"])

In [ ]:
G_taper1.plot("field_power")

In [ ]:
taper1.load_particles()
pbeg1 = taper1.particles['beginning']
pend1 = taper1.particles['end']

In [ ]:
pend1.plot("z", "energy", bins=200)

In [ ]:
# Scan peak current

G_scan_1 = G_taper1
current_list_1 = np.linspace(500, 3000, 5)
scan_output_1 = []

for current in current_list_1:
    G_scan_1.input.main.beam.current = current
    G_scan_1.verbose = False
    scan_output_1.append(G_scan_1.run())

In [ ]:
fig, ax = plt.subplots()
for k, g in zip(current_list_1, scan_output_1):
    x = g.stat("zplot")
    y = g.stat("power")
    ax.plot(x, y / 1e6, label=f"{k:0.1f}")
ax.set_yscale("log")
ax.set_xlabel(r"$z$ (m)")
ax.set_ylabel("power (MW)")
plt.legend(title="Peak Current(A)")

In [ ]:
fig, ax = plt.subplots()
y = np.array([g.stat("power")[-1] for g in scan_output_1])
ixbest = y.argmax()
Gbest = scan_output_1[ixbest]
current_best_1 = current_list_1[ixbest]
ybest = y[ixbest]
ax.plot(current_list_1, y / 1e6)
ax.scatter(current_best_1, ybest / 1e6, marker="*", label=rf"$k$= {current_best_1:0.1f} 1/m$^2$")
ax.set_ylabel(r"end power $(MW)$")
ax.set_xlabel(r"Peak Current $(A)$")
plt.legend()

## Trying more agressive linear taper. Check file **LCLS00-TAPER2.in** for lattice

In [ ]:
# more agressive taper
G_taper2 = Genesis4("examples/LCLSII-TAPER2.in")
G_taper2.verbose = True
G_taper2.input

In [ ]:
taper2 = G_taper1.run()

In [ ]:
taper2.lattice.plot();

In [ ]:
# Scan peak current

G_scan_2 = G_taper2
current_list_2 = np.linspace(500, 3000, 5)
scan_output_2 = []

for current in current_list_2:
    G_scan_2.input.main.beam.current = current
    G_scan_2.verbose = False
    scan_output_2.append(G_scan_2.run())

In [ ]:
fig, ax = plt.subplots()
for k, g in zip(current_list_2, scan_output_2):
    x = g.stat("zplot")
    y = g.stat("power")
    ax.plot(x, y / 1e6, label=f"{k:0.1f}")
ax.set_yscale("log")
ax.set_xlabel(r"$z$ (m)")
ax.set_ylabel("power (MW)")
plt.legend(title=r"Peak Current $(A)$")

In [ ]:
fig, ax = plt.subplots()
y = np.array([g.stat("power")[-1] for g in scan_output_2])
ixbest = y.argmax()
Gbest = scan_output_2[ixbest]
current_best_2 = current_list_2[ixbest]
ybest = y[ixbest]
ax.plot(current_list_2, y / 1e6)
ax.scatter(current_best_2, ybest / 1e6, marker="*", label=rf"$k$= {current_best_2:0.1f} 1/m$^2$")
ax.set_ylabel(r"end power $(MW)$")
ax.set_xlabel(r"Peak Current $(A)$")
plt.legend()

In [ ]:
G_taper2.plot(["beam_xsize", "beam_ysize", "field_xsize", "field_ysize"])

In [ ]:
G_taper2.plot("field_power")

In [ ]:
taper2.load_particles()
pbeg2 = taper2.particles['beginning']
pend2 = taper2.particles['end'] 

In [ ]:
pbeg2.plot("z", "energy", bins=200)

In [ ]:
pend2.plot("z", "energy", bins=200)

# Summary (Taper)

In [ ]:
no_taper_power = output.stat("power")
taper1_power = taper1.stat("power")
taper2_power = taper2.stat("power")
y_list = [no_taper_power, taper1_power, taper2_power]
label = ["no_taper_power", "taper1_power", "taper2_power"]
x = g.stat("zplot")

fig, ax = plt.subplots()
for i, (y, ylabel) in enumerate(zip(y_list, label)):
    ax.plot(x, y, label=ylabel)
ax.set_yscale("log")
ax.legend()

# Energy spread

## Scanning through various values of delgam. 

delgam = 1 corresponds to rms energy spread of 0.511 MeV

In [ ]:
# Scan energy spread (delgam)

G_scan = G
delgam_list = np.linspace(0, 5, 5)     #delgam = 1 is rms energy spreak of 0.511 MeV
scan_output_delgam = []

for delgam in delgam_list:
    G_scan.input.main.beam.delgam = delgam
    G_scan.verbose = False
    scan_output_delgam.append(G_scan.run())

In [ ]:
fig, ax = plt.subplots()
for k, g in zip(delgam_list, scan_output_delgam):
    x = g.stat("zplot")
    y = g.stat("power")
    ax.plot(x, y, label=f"{k*0.511:0.1f}MeV")
ax.set_yscale("log")
ax.set_xlabel(r"$z$ (m)")
ax.set_ylabel("power (MW)")
plt.legend(title=r"Energy Spread")

In [ ]:
fig, ax = plt.subplots()
y = np.array([g.stat("power")[-1] for g in scan_output_delgam])
ixbest = y.argmax()
Gbest = scan_output_delgam[ixbest]
delgam_best = delgam_list[ixbest]
ybest = y[ixbest]
ax.plot(delgam_list, y / 1e6)
ax.scatter(delgam_best, ybest / 1e6, marker="*", label=rf"{delgam_best*0.511:0.1f} MeV")
ax.set_ylabel(r"end power $(MW)$")
ax.set_xlabel(r"Energy Spread")
plt.legend()

In [ ]:
scan_output_delgam[0].load_particles()
pbeg_d = scan_output_delgam[0].particles['beginning']
pend_d = scan_output_delgam[0].particles['end']
pend_d.plot("z", "energy", bins=200)

# Long Undulator

In [ ]:
G_long = Genesis4("examples/lcls-II-long.in")
G_long.verbose = True

In [ ]:
output_long = G_long.run()

In [ ]:
output_long.plot("power", yscale="log")

In [ ]:
output.plot("power", yscale="log")